<a href="https://colab.research.google.com/github/joaopgioio/projetoDetectObjetoYolo/blob/master/Projeto_Dio_Real_Time_Detect_Objeto_Image_Yolo_FalarObjetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --upgrade --quiet ultralytics
!pip install gTTS
#!pip install googletrans==4.0.0-rc1
!pip install --upgrade googletrans
!pip install deep-translator
!pip install playsound
!pip install pygobject
!pip install pyglet
!apt-get install -y xvfb # Install Xvfb
!pip install pyvirtualdisplay # Install a Python wrapper for Xvfb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 948.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=1a09aa1df0edd057ba1953505762dbd4611f3ce302d9bfcec2b35133559a272e
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d

In [8]:
from base64 import b64decode, b64encode
from google.colab.output import eval_js
from IPython.display import display, Javascript, Audio
from PIL import Image
from ultralytics import YOLO
from ultralytics.engine.results import Results
import numpy as np
from gtts import gTTS
from playsound import playsound
import io
import pyglet  # Import pyglet for audio playback
from pyvirtualdisplay import Display
import time # Importa o módulo 'time'
from googletrans import Translator
from deep_translator import GoogleTranslator

# Change the variable name to avoid overwriting the display function
virtual_display = Display(visible=0, size=(1024, 768))
virtual_display.start()


MODEL_NAMES = ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt']
PRE_TRAINED_MODEL = YOLO(MODEL_NAMES[0])
IMG_SHAPE = [640, 480]
IMG_QUALITY = 0.8

In [9]:
def start_stream():
    js = Javascript(f'''
    const IMG_SHAPE = {IMG_SHAPE};
    const IMG_QUALITY = {IMG_QUALITY};
    ''' + '''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
        stream.getVideoTracks()[0].stop();
        video.remove();
        div.remove();
        video = null;
        div = null;
        stream = null;
        imgElement = null;
        captureCanvas = null;
        labelElement = null;
    }

    function onAnimationFrame() {
        if (!shutdown) {
            window.requestAnimationFrame(onAnimationFrame);
        }
        if (pendingResolve) {
            var result = "";
            if (!shutdown) {
                captureCanvas.getContext('2d').drawImage(video, 0, 0, IMG_SHAPE[0], IMG_SHAPE[1]);
                result = captureCanvas.toDataURL('image/jpeg', IMG_QUALITY)
            }
            var lp = pendingResolve;
            pendingResolve = null;
            lp(result);
        }
    }

    async function createDom() {
        if (div !== null) {
            return stream;
        }

        div = document.createElement('div');
        div.style.border = '2px solid black';
        div.style.padding = '3px';
        div.style.width = '100%';
        div.style.maxWidth = '600px';
        document.body.appendChild(div);

        const modelOut = document.createElement('div');
        modelOut.innerHTML = "<span>Status: </span>";
        labelElement = document.createElement('span');
        labelElement.innerText = 'No data';
        labelElement.style.fontWeight = 'bold';
        modelOut.appendChild(labelElement);
        div.appendChild(modelOut);

        video = document.createElement('video');
        video.style.display = 'block';
        video.width = div.clientWidth - 6;
        video.setAttribute('playsinline', '');
        video.onclick = () => { shutdown = true; };
        stream = await navigator.mediaDevices.getUserMedia(
            {video: { facingMode: "environment"}});
        div.appendChild(video);

        imgElement = document.createElement('img');
        imgElement.style.position = 'absolute';
        imgElement.style.zIndex = 1;
        imgElement.onclick = () => { shutdown = true; };
        div.appendChild(imgElement);

        const instruction = document.createElement('div');
        instruction.innerHTML =
            '<span style="color: red; font-weight: bold;">' +
            'When finished, click here or on the video to stop this demo</span>';
        div.appendChild(instruction);
        instruction.onclick = () => { shutdown = true; };

        video.srcObject = stream;
        await video.play();

        captureCanvas = document.createElement('canvas');
        captureCanvas.width = IMG_SHAPE[0]; //video.videoWidth;
        captureCanvas.height = IMG_SHAPE[1]; //video.videoHeight;
        window.requestAnimationFrame(onAnimationFrame);

        return stream;
    }
    async function takePhoto(label, imgData) {
        if (shutdown) {
            removeDom();
            shutdown = false;
            return '';
        }

        var preCreate = Date.now();
        stream = await createDom();

        var preShow = Date.now();
        if (label != "") {
            labelElement.innerHTML = label;
        }

        if (imgData != "") {
            var videoRect = video.getClientRects()[0];
            imgElement.style.top = videoRect.top + "px";
            imgElement.style.left = videoRect.left + "px";
            imgElement.style.width = videoRect.width + "px";
            imgElement.style.height = videoRect.height + "px";
            imgElement.src = imgData;
        }

        var preCapture = Date.now();
        var result = await new Promise((resolve, reject) => pendingResolve = resolve);
        shutdown = false;

        return {
            'create': preShow - preCreate,
            'show': preCapture - preShow,
            'capture': Date.now() - preCapture,
            'img': result,
        };
    }
    ''')
    display(js)

def take_photo(label, img_data):
    data = eval_js(f'takePhoto("{label}", "{img_data}")')
    return data

In [10]:
def js_response_to_image(js_response) -> Image.Image:
    _, b64_str = js_response['img'].split(',')
    jpeg_bytes = b64decode(b64_str)
    image = Image.open(io.BytesIO(jpeg_bytes))
    return image

def turn_non_black_pixels_visible(rgba_compatible_array: np.ndarray) -> np.ndarray:
    rgba_compatible_array[:, :, 3] = (rgba_compatible_array.max(axis=2) > 0).astype(int) * 255
    return rgba_compatible_array

def black_transparent_rgba_canvas(w, h) -> np.ndarray:
    return np.zeros([w, h, 4], dtype=np.uint8)

def draw_annotations_on_transparent_bg(detection_result: Results) -> Image.Image:
    black_rgba_canvas = black_transparent_rgba_canvas(*detection_result.orig_shape)
    transparent_canvas_with_boxes_invisible = detection_result.plot(font='verdana', masks=False, img=black_rgba_canvas)
    transparent_canvas_with_boxes_visible = turn_non_black_pixels_visible(transparent_canvas_with_boxes_invisible)
    image = Image.fromarray(transparent_canvas_with_boxes_visible, 'RGBA')
    return image

In [ ]:
start_stream()
img_data = ''
while True:
    js_response = take_photo('Capiturando...', img_data)
    if not js_response:
        break
    captured_img = js_response_to_image(js_response)

    def traduzir_lista(lista_ingles):
      """
      Traduz uma lista de palavras do inglês para o português.

      Args:
        lista_ingles: Uma lista de palavras em inglês.

      Returns:
        Uma lista de palavras em português.
      """

      tradutor = Translator()
      lista_portugues = []
      for palavra in lista_ingles:
        #traducao = tradutor.translate(palavra, dest='pt')
        traducao = GoogleTranslator(source='en', target='pt').translate(palavra)
        lista_portugues.append(traducao)
      return lista_portugues


    for detection_result in PRE_TRAINED_MODEL(source=np.array(captured_img), verbose=False):
        annotations_img = draw_annotations_on_transparent_bg(detection_result)

        results = PRE_TRAINED_MODEL(source=np.array(captured_img), verbose=False)

        # Extract detected object names
        objetos_detectados = [result.names[int(c)] for result in results for c in result.boxes.cls]

        # Crie uma frase com os objetos detectados

        if isinstance(objetos_detectados, list):
          objetos_detectados_portugues = traduzir_lista(objetos_detectados)
          #print(objetos_detectados_portugues)  # Output: ['gato', 'cão', 'pessoa']
        else:
          frase_objetos = "No objects detected" # Provide a message when no objects are found

        #print(objetos_detectados)
        #print(objetos_detectados_portugues)

        # Crie uma frase com os objetos detectados
        frase_objetos = "" + ", ".join(objetos_detectados)
        frase_objetos_portugues = "" + ", ".join(objetos_detectados_portugues)


        # Crie o áudio usando gTTS
        tts = gTTS(text=frase_objetos, lang='pt')
        tts_portugues = gTTS(text=frase_objetos_portugues, lang='pt')

        # Salve o áudio em um arquivo
        tts.save("objetos_detectados.mp3")
        tts_portugues.save("objetos_detectados_portugues.mp3")

        def falar_objetos():
            # Use the display module from IPython to play the audio
            display(Audio("objetos_detectados.mp3", autoplay=True))

            # Aguarda 2 segundos antes de continuar
            time.sleep(len(objetos_detectados)+2)
            # Use the display module from IPython to play the audio
            display(Audio("objetos_detectados_portugues.mp3", autoplay=True))

            # Aguarda 2 segundos antes de continuar
            time.sleep(5)

        # Salve o áudio em um arquivo temporário
        #with open('temp_audio.mp3', 'wb') as f:
        #    tts.write_to_fp(f)

        # Reproduza o áudio do arquivo temporário
        #playsound('temp_audio.mp3')



        # Use pyglet to play the audio file
        #music = pyglet.media.load('objetos_detectados.mp3', streaming=False)
        #music.play()
        #pyglet.app.run()

        # Close the window after audio playback
        #window.close()




        with io.BytesIO() as buffer:
            annotations_img.save(buffer, format='png')
            img_as_base64_str = str(b64encode(buffer.getvalue()), 'utf-8')
            img_data = f'data:image/png;base64,{img_as_base64_str}'
    falar_objetos()

<IPython.core.display.Javascript object>